In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


np.random.seed(42)

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df

## **Basic Data Exploration**

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns = [name.replace(' ', '_') for name in df.columns]
df.columns

In [ ]:
plt.rcParams["figure.figsize"] = [12, 9]

In [ ]:
df.plot(subplots=True)
plt.show()

## **Correlation Matrix**

In [ ]:
import seaborn as sns

In [ ]:
corr = df.corr()
ax = sns.heatmap(
     corr, 
     vmin=-1, vmax=1, center=0,
     cmap=sns.diverging_palette(20, 220, n=200),
     square=True,
     annot=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
plt.plot()

In [ ]:
X = df.loc[:, df.columns[:len(df.columns)-1]]
X

In [ ]:
y = df[['quality']]
y

## **Testing Various Regression Models**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = StandardScaler().fit_transform(X)  # normalization data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
y_hat = reg.predict(X_test)

In [ ]:
r2_score(y_test, y_hat)

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
reg1 = RidgeCV(alphas=[0.001, 0.01, 0.1], cv=10).fit(X, y)
reg1.score(X, y)

In [ ]:
reg1.get_params()

In [ ]:
reg1.alpha_

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
reg2 = ElasticNetCV(cv=10, random_state=42).fit(X, y)
reg2.score(X, y)

In [ ]:
reg2.alpha_

Best $R^2$ score was given by linear model without modifications

## **Classification with Logistic Regression**

We have 6 classes, so we can predict class for red-wine in this dataset. It is possible to apply multiclass classifier, but resonable is to split marks into good and bad, as author of this dataset advices. If $mark < 6.5$ then wine quality is bad, otherwise good.

In [ ]:
y.quality.unique()

In [ ]:
y_lgs = y.quality.apply(lambda x: 1 if x > 6.5 else 0).to_frame()
y_lgs

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_lgs,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(solver='lbfgs', random_state=4)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_hat = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

**Logistic regression classifier score:**

In [ ]:
accuracy_score(y_test, y_hat)

This result is way better than we have with regression models.

## **Lets try Random Forest**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_hat = clf.predict(X_test)

**Random Forest Classifier score**

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
df.columns

In [ ]:
feature_importance = pd.Series(clf.feature_importances_, index=df.columns[:11]).sort_values(ascending=False)
feature_importance

In [ ]:
sns.barplot(x=feature_importance, y=feature_importance.index)
plt.xlabel('Feature importance Score')
plt.ylabel('Features')
plt.title('Visualizing Important Features')
plt.legend()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [ ]:
rf = RandomForestRegressor(random_state=42)

In [ ]:
print('Parameters currently in use: \n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]

In [ ]:
max_features = ['auto', 'sqrt']

In [ ]:
max_depth = [int(x) for x in np.linspace(2, 14, 7)]
max_depth.append(None)

In [ ]:
min_samples_split = [2, 5, 10]

In [ ]:
min_samples_leaf = [1, 2, 4]

In [ ]:
bootstrap = [True, False]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
base_model = RandomForestRegressor(n_estimators=10, random_state=42)

In [ ]:
base_model.fit(X_train, y_train)

In [ ]:
y_hat = base_model.predict(X_test)
base_accuracy = r2_score(y_test, y_hat)
print('Accuracy of base model:\n', base_accuracy)

In [ ]:
best_random = rf_random.best_estimator_
y_hat = best_random.predict(X_test)
random_accuracy = r2_score(y_test, y_hat)
print('Accuracy of random model:\n', random_accuracy)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

## **Practice with GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [8, 10, 12, 14],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [ ]:
rf = RandomForestRegressor()

In [ ]:
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
y_hat = best_grid.predict(X_test)
grid_accuracy = r2_score(y_test, y_hat)
print('Accuracy of grid model:\n', grid_accuracy)

## **Bring Neural Networks to work**

Random Forest performs pretty good, but lets try another powerful model - parceptron. Maybe we can built better model with basic neural network, or ensemble of parceptrons.

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
nnrg = MLPRegressor(solver='adam', learning_rate='adaptive', random_state=42,
                    max_iter=500)

In [ ]:
nnrg.fit(X_train, y_train)

In [ ]:
nnrg.score(X_test, y_test)

Basic Perceptron performed pretty well, at least better than **_Linear Regression_** model, but worse than _Random Forest_, lets try to find better parameters for this model. And algorithm is not converged.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [200, 300, 500, 1000, 1500]
}

In [ ]:
nn = MLPRegressor()

In [ ]:
random_search = GridSearchCV(estimator=nn,
                             param_grid=param_grid,
                             cv=3,
                             n_jobs=-1,
                             verbose=2)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_params_

In [ ]:
random_search.score(X_test, y_test)

In [ ]:
abs(random_search.score(X_test, y_test) - nnrg.score(X_test, y_test))*100

With randomized search model preformed worse than with basic parameters. But maybe it can do better predictions. So, with this regressor lets try to ensamble perceptrons.

For this purpose I decided to try technic that used in Random Forest - **Bagging**, and let's use 300 models, as we used in Random Forest above.

In [ ]:
from sklearn.ensemble import BaggingRegressor

I set **_max_iter_** to 1500, because I see that minimum is not found by amout of iterations equal to 700 and 1000.

In [ ]:
nnb = MLPRegressor(activation='tanh',
                   alpha=0.0001,
                   hidden_layer_sizes=(100,),
                   learning_rate='adaptive',
                   solver='adam',
                   max_iter=1500,
                   random_state=42)

In [ ]:
clf = BaggingRegressor(base_estimator=nnb,
                       n_estimators=300,
                       random_state=42,
                       verbose=2)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

As expected ensemble of perceptrons works worse than ensemble of decision trees, and I think that random forest is best algorithm for this task, but probably Boosting Tree(XGBoost) can perform better.